# ODIMA Machine Learning Classifier

In [ ]:
###setting environment up
%reset -f
import pandas as pd
import numpy as np
import os
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cross_decomposition import CCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
import scipy.stats as stats
import plotly.express as px
import plotly.io as pio


## Loading data and creating a combined dataset for analysis

In [ ]:
# %% Load your data
file_path = 'mODIMA _Neuro_Scores.xlsx'
data = pd.read_excel(file_path)

#print(data.head())
#type(data)

## converting the category column to be binary
#data['Category'] = data['Category'].replace({'HC': 0, 'MCI': 1}) ## replace is no longer in use in the future
data['Category'] = np.where(data['Category'] == 'HC', 0, 1)


##checking basic statistics of the dataset
data.describe()

## create a barona index score using all the variables before unnecessary variables are removed
#first change the variables to the format needed for the index inlcuding sex and race
data.loc[data['Sex'] == 'F' , 'Sex'] = 0
data.loc[data['Sex'] == 'M' , 'Sex'] = 1

data['Sex'] = data['Sex'].astype(int)

#Replace data with strings based on condition
data['Race'] = np.where(data['Race'] == 'White', 0, 1)

#make the category colum an integer
#data['Category'] = data['Category'].astype(int)
#print(data.dtypes)

# Define a function to calculate Barona Index
def calculate_barona_index(row):
    predicted_iq = 101.2 + (0.27 * row['Age']) + (0.54 * row['Education']) + (0.64 * row['Sex']) - (3.47 * row['Race'])
    return predicted_iq

# Apply the function to the DataFrame
data['Barona_Index'] = data.apply(calculate_barona_index, axis=1)

##move barona index to the front
barona_insert = data.pop('Barona_Index')
data.insert(2,'Barona_index',barona_insert)

##removing non numeric and other columns
data.drop(['DOB','MRN','Sex','Education','Race','Date (Neuro)','Visual Verbal Test','Letter Number Sequencing'],axis = 1 , inplace = True)

#drop the unnamed column at the end of the dataset
data.drop(data.columns[39], axis=1,inplace=True)

##removing rows where there are no olfactory scores
data = data.dropna(subset=['Threshold', 'Identification'], how='any')
original_columns =pd.DataFrame(data.columns)
original_columns =original_columns.T
original_columns.columns = original_columns.iloc[0]  # Set first row as column names
original_columns= original_columns[1:].reset_index(drop=True)  # Drop the first row and reset index


#### we notice that there are a few cases where the olfactory scores are zero
## we want to get rid of these particiants
dropped_rows = data[data.Identification < 1]
#print(dropped_rows)

##dropping the 4 selected participants - ODIMA002_F1,0047,0053 and 1001_F1
data.drop(data[data.Identification < 1].index, inplace=True)

data_scaled = data.drop(data.columns[[0, 1]], axis=1)

# Reset the index of the DataFrame as we have droppe some data earlier
data.reset_index(drop=True, inplace=True)
data_scaled.reset_index(drop=True, inplace=True)

## create a new dataframe with the dropped columns so it can be concatanated
dropped_columns = data.iloc[:, [0, 1]]

#print(data_scaled.dtypes)

In [ ]:
#create a dataset with all variables standardized and one with all raw scores
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_scaled)  # Standardized data for clustering

##taking the numpy array datascaled and making it a pandas dataframe
data_scaled = pd.DataFrame(data_scaled)  # Convert to DataFrame if it's not already

# Concatenate the dropped columns back to the front of the data_scaled DataFrame
data_scaled = pd.concat([dropped_columns, data_scaled], axis=1)
##renaming the dataframe columns with the string scaled so it can be used to create a master data table
data_scaled.columns = [f"{col}_scaled" for col in original_columns]

final_data = pd.concat([data, data_scaled.iloc[:,2:38]], axis=1)


## Normality Testing

In [ ]:
#Create a function to test normality for multiple columns in the dataset and plot values to look at the distribution
def check_normality(data):
    results = []

    for column in data.select_dtypes(include=[np.number]).columns:  # Select numeric columns only
        print(f"\n📊 Checking Normality for ➝ {column}")

        # Histogram + KDE
        plt.figure(figsize=(8, 4))
        sns.histplot(data[column], kde=True, bins=30)
        plt.title(f'Histogram & KDE for ➝ {column}')
        plt.show()

        # Q-Q Plot
        plt.figure(figsize=(6, 6))
        stats.probplot(data[column].dropna(), dist="norm", plot=plt)
        plt.title(f'Q-Q Plot for ➝ {column}')
        plt.show()

        # Shapiro-Wilk Test
        shapiro_stat, shapiro_p = stats.shapiro(data[column].dropna()) if len(data[column]) < 5000 else (None, None)

        # Kolmogorov-Smirnov Test
        ks_stat, ks_p = stats.kstest(data[column].dropna(), 'norm')

        # Anderson-Darling Test
        anderson_stat = stats.anderson(data[column].dropna(), dist='norm')

        # Skewness & Kurtosis
        skewness = stats.skew(data[column], nan_policy='omit')
        kurt = stats.kurtosis(data[column], nan_policy='omit')

        # Save results
        results.append({
            'Column': column,
            'Shapiro P-Value': shapiro_p,
            'KS P-Value': ks_p,
            'Anderson Stat': anderson_stat.statistic,
            'Skewness': skewness,
            'Kurtosis': kurt
        })

        # Print results
        print(f"Shapiro-Wilk Test: p = {shapiro_p} {'✅ Normal' if shapiro_p and shapiro_p > 0.05 else '❌ Not Normal'}")
        print(f"KS Test: p = {ks_p} {'✅ Normal' if ks_p > 0.05 else '❌ Not Normal'}")
        print(f"Anderson-Darling Statistic: {anderson_stat.statistic}")
        print(f"Skewness: {skewness}, Kurtosis: {kurt}")

    return pd.DataFrame(results)

# Run the function
normality_results = check_normality(final_data)

# Display summary results
print("\n📌 Summary of Normality Tests:")
print(normality_results)




# K- Means Clustering Analysis

In [ ]:

##Create a function to draw an elbow plot based on data to determine k
max_k = 10 ## determine maximum k here 
##define the function below
def optimise_k_means(data, max_k):
    means = []
    inertias = []

    for k in range(1, max_k + 1):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data)

        means.append(k)
        inertias.append(kmeans.inertia_)

    return means, inertias



##generate the data based on the above created function and maximum number of clusters

means, inertias = optimise_k_means(data.iloc[:,2:38], max_k)

## generate elbow plot

plt.figure(figsize=(10, 5))
plt.plot(means, inertias, 'o-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

## we will use either 2,3 and 4 cluster based on the elbow plot too see how the classification goes!


### 3 Kmeans cluster - All variables used in PCA 

In [ ]:
# Perform PCA on the data to reduce dimensionality and allow clasifier algorithm to run
## we use all the variables other than category to make the principal components
pca = PCA(n_components=2)  # we set the PCA components to be 2 
X_pca = pca.fit_transform(data_scaled.iloc[:,2:39])

# Convert to the new PCA data into a data fram as it was a numpy array
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])

print("Explained Variance Ratio:", pca.explained_variance_ratio_)
print("Cumulative Explained Variance:", np.cumsum(pca.explained_variance_ratio_))

##plot the principal components
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1],c=X_pca[:, 0], cmap='viridis', edgecolors='k', alpha=0.8)
plt.colorbar(label="PC1 Value")  # Adds a color legend
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("Principal Component Analyis Biplot - Standardized data")
plt.show()

In [ ]:
## check the PCA component loadings to understand which variables are the most important 

# Assuming PCA is already fitted
loadings = pca.components_

# Convert to DataFrame for better readability
loading_df = pd.DataFrame(loadings, columns=data_scaled.columns[2:39], index=['PC1', 'PC2'])
#print(loading_df)


plt.figure(figsize=(12, 6))
sns.heatmap(loading_df, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)
plt.title("PCA Loadings (Feature Contributions to PCs)")
plt.show()

for i in range(2):  # Loop over PC1 and PC2
    sorted_features = loading_df.iloc[i, :].abs().sort_values(ascending=False)
    print(f"Top 5 Features for {loading_df.index[i]}:\n{sorted_features.head(5)}\n")

In [ ]:
# We run a k means cluster based to classify the raw data!
#optimal_k = 3  # based on elbow diagram we set the number of clusters to be 3!
#kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10) ## we set a random state for reproduceability!
#clusters = kmeans.fit_predict(X_pca)

# Add cluster labels to PCA DataFrame
#pca_df['Cluster'] = clusters

### plot principal compponents classified by clusters!!
#plt.figure(figsize=(8,6))
#plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['Cluster'], cmap='viridis', alpha=1)
#plt.xlabel("Principal Component 1")
#plt.ylabel("Principal Component 2")
#plt.title(f"K-Means Clustering (k={optimal_k}) on PCA")
#plt.colorbar(label="Cluster")
#plt.show()

In [ ]:

# We run a k means cluster based to classify the raw data!
optimal_k = 3  # based on elbow diagram we set the number of clusters to be 3!
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10) ## we set a random state for reproduceability!
clusters = kmeans.fit_predict(X_pca)

# Add cluster labels to PCA DataFrame
pca_df['Cluster'] = clusters

##add the ODIMA_ID and original classification to the data frame so i can plot 
ODIMA_CAT_Master = data_scaled[['Study ID_scaled','Category_scaled']]  #excract columns from DataFrame

# Rename the columns to the original names
ODIMA_CAT_Master.columns = ['Study ID', 'Category']

# Insert the columns into pca_df at the desired position
pca_df.insert(2, 'Study ID', ODIMA_CAT_Master['Study ID'])
pca_df.insert(3, 'Category', ODIMA_CAT_Master['Category'])
# Ensure Category is numeric or mapped to numbers
pca_df['Category'] = pca_df['Category'].astype(int)


In [ ]:
## basic plot which is no longer in use due to a color cordinated one being used

# Create scatter plot
#plt.figure(figsize=(8, 6))
#plt.scatter(pca_df['PC1'], pca_df['PC2'], 
#c=pca_df['Cluster'], cmap= 'viridis', alpha=1)
# Add labels and title
#plt.xlabel("Principal Component 1")
#plt.ylabel("Principal Component 2")
#plt.title(f"K-Means Clustering (k={3}) on PCA")  # Update optimal_k accordingly
#plt.colorbar(label="Cluster")
#plt.show()

In [ ]:
## create a basic plot with colors assigned to cluster for easier recognition
# Custom colors for each cluster are assigned
cluster_colors = {0: 'green', 1: 'red', 2: 'yellow'}
# names are assigned to each cluster
cluster_names = {0: 'Cluster 0 - Mainly HC', 1: 'Cluster 1 - Mainly MCI', 2: 'Cluster 2 - Mixed bag'}

# Create scatter plot
plt.figure(figsize=(8, 6))
for category in np.unique(pca_df['Cluster']):
    category_data_1 = pca_df[pca_df['Cluster'] == category]
    plt.scatter(category_data_1['PC1'], category_data_1['PC2'], 
                color=cluster_colors[category], label=cluster_names.get(category, f'Category {category}'), alpha=0.7)

# Add labels and title
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA - K Means=3 Classifier")
plt.legend()
plt.show()


In [ ]:
## plotly graph for the above so that it is interactive
# Set the default renderer to open up in a browser
pio.renderers.default = "browser"

# Custom colors for each Cluster
cluster_colors = {0: 'green', 1: 'red', 2: 'yellow'}
# Mapping Cluster numbers to new names
cluster_names_n = {0: 'Cluster 0- Mainly HC', 1: 'Cluster 1- Mainly MCI', 2: 'Cluster 2- Mixed bag'}

# Ensure 'Cluster' is treated as a categorical variable for the plots
pca_df['Cluster'] = pca_df['Cluster'].astype('category')

# Set a template for the plot
pio.templates.default = "simple_white"  # Replace as needed

# Create an interactive scatter plot with Plotly
fig = px.scatter(
    pca_df, 
    x='PC1', 
    y='PC2', 
    color='Cluster', 
    hover_data=['Study ID'],  # Display Study ID on hover to see which participants need to be focused on
    title="PCA - New Kmeans 3 Classifier",
    labels={
        'PC1': 'Principal Component 1',
        'PC2': 'Principal Component 2',
        'Category': 'New Clusters'
    },
    color_discrete_map=cluster_colors  # Set custom colors for categories
)

# Update legend labels using the cluster_names_n dictionary
fig.for_each_trace(lambda t: t.update(name=cluster_names_n.get(int(t.name), t.name)))

fig.show()

### Plot the Principal Components based on their original Group in the ODIMA study

In [ ]:
### plot principal compponents classified by the original ODIMA study group
#no longer in use as new plot is made with consistent color coding

#plt.figure(figsize=(8,6))
#plt.scatter(pca_df['PC1'], pca_df['PC2'], c=pca_df['Category'], cmap='viridis', alpha=1)


# Add text labels for each point using Study ID ## removed as it makes the plot impossible to see
#for i, txt in enumerate(pca_df['Study ID']):
    #plt.annotate(txt, (pca_df['PC1'].iloc[i], pca_df['PC2'].iloc[i]), 
                 #fontsize=8, alpha=0.7)

#plt.xlabel("Principal Component 1")
#plt.ylabel("Principal Component 2")
#plt.title("PCA - Original ODIMA Category Classification")
#plt.colorbar(label="ODIMA Category")
#plt.show()



In [ ]:
# Create custom colors for each ODIMA category
category_colors = {0: 'Green', 1: 'red'}
#name the columns
category_names = {0: 'Healthy Controls', 1: 'Mild Cognitive Impairment'}

# Create scatter plot
plt.figure(figsize=(8, 6))
for category in np.unique(pca_df['Category']):
    category_data = pca_df[pca_df['Category'] == category]
    plt.scatter(category_data['PC1'], category_data['PC2'], 
                color=category_colors[category], label=category_names.get(category, f'Category {category}'), alpha=0.7)

# Add labels and title
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA - Original ODIMA Category Classification")
plt.legend()
plt.show()

In [ ]:
## plotly graph for the above so that it is interactive , colors have been set to be consistent

# Set the default renderer
pio.renderers.default = "browser"

# Custom colors for Healthy Controls (0) and MCI (1)
category_colors_n = {0: 'green', 1: 'red'}
# Mapping category numbers to new names
category_names_n = {0: 'Healthy Controls', 1: 'Mild Cognitive Impairment'}
# Ensure 'Category' is treated as a categorical variable
pca_df['Category'] = pca_df['Category'].astype('category')

# Set the default template to a built-in theme
pio.templates.default = "simple_white"  # Replace as needed

# Create an interactive scatter plot with Plotly
fig = px.scatter(
    pca_df, 
    x='PC1', 
    y='PC2', 
    color='Category', 
    hover_data=['Study ID'],  # Display Study ID on hover
    title="PCA - Original ODIMA Category Classification",
    labels={
        'PC1': 'Principal Component 1',
        'PC2': 'Principal Component 2',
        'Category': 'ODIMA Category'
    },
    color_discrete_map=category_colors_n  # Set custom colors for categories
)
# Update legend labels using the category_names dictionary
fig.for_each_trace(lambda t: t.update(name=category_names_n.get(int(t.name), t.name)))

fig.show()

In [ ]:
##next i take the new clusters and then add them back to the original dataset for further analysis!
#print(len(data))
#print(len(final_data))
#print(len(pca_df))
#print(pca_df['Cluster'].isna().sum()) 

##creating a new data frame with the cluster information for analaysis
#dataset is named based on number of kmeans clustering used
cluster3 = pd.concat([data , pd.DataFrame({'Cluster': pca_df['Cluster'].values})], axis=1)

##next i want to move the cluster columns to where the initial subject category column is to see how the data has been classified and make analysis easier!
 
# Move column 'Cluster' to be the first column
col = cluster3.pop('Cluster')  # Remove 'C' from DataFrame
cluster3.insert(2, 'Cluster', col)  # Insert 'C' at index 0

## CLuster 0
df_filtered_3_0 = cluster3[cluster3['Cluster'] == 0]
#print(df_filtered_3_0)

## count the number of zero cluster by category
result_3_0 = df_filtered_3_0.groupby('Category')['Cluster'].count()
result_3_0 = result_3_0.reset_index(name='Count')

##creating a perecntage to see how many HC and MCI are being classified as cluster 0
result_3_0['Percentage']= (result_3_0['Count'] / result_3_0['Count'].sum())*100
print(result_3_0)


## Cluster 1
df_filtered_3_1 = cluster3[cluster3['Cluster'] == 1]
#print(df_filtered_3_1)

## count the number of zero cluster by category
result_3_1= df_filtered_3_1.groupby('Category')['Cluster'].count()
result_3_1 = result_3_1.reset_index(name='Count')

##creating a perecntage to see how many HC and MCI are being classified as cluster 0
result_3_1['Percentage']= (result_3_1['Count'] / result_3_1['Count'].sum())*100
print(result_3_1)

###Cluster 2
df_filtered_3_2 = cluster3[cluster3['Cluster'] == 2]
#print(df_filtered_3_2)

## count the number of zero cluster by category
result_3_2= df_filtered_3_2.groupby('Category')['Cluster'].count()
result_3_2 = result_3_2.reset_index(name='Count')

##creating a perecntage to see how many HC and MCI are being classified as cluster 0
result_3_2['Percentage']= (result_3_2['Count'] / result_3_2['Count'].sum())*100
print(result_3_2)
